In [1]:
import pandas as pd
import boto3
from io import BytesIO
import json
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from  src.calc_primas import calcular_edad
from src.calc_primas import obtener_lista_nombre_bases
from src.calc_primas import obtener_base_parametros
from src.calc_primas import obtener_base_cuotas
from src.calc_primas import obtener_base_emisiones
from src.calc_primas import obtener_base_historico
from src.calc_primas import obtener_nombre_mes
from src.calc_primas import obtener_parametros_forma_pago
from src.calc_primas import obtener_descuento_comision
from src.calc_primas import obtener_nombre_cobertura
from src.calc_primas import generar_memoria_calculo
from src.calc_primas import creacion_cotizacion_dict
from src.pdf_utils import cargar_dict_cotizacion
from src.pdf_utils import convertir_campo_a_float
from src.pdf_utils import convertir_campo_a_fecha
from src.pdf_utils import generar_pdf_cotizacion
from src.pdf_utils import obtener_nombres_empresas
from src.pdf_utils import draw_header_bar
from src.pdf_utils import truncate_text
from src.pdf_utils import draw_table_with_borders
from src.pdf_utils import draw_single_column_table


### PIPELINE SAGEMAKER

In [2]:
# Pipline SAGEMAKER
# 1. Cargar base de datos limpios
# Nombre del bucket
bucket_name = "itam-analytics-grb"
s3 = boto3.client("s3")
# Rutas
ruta_calculo = "coco/data/cleaned/calculo" # Ruta de la base de datos de asegurados para crear cotizaciones y memoria de calculo
ruta_parametros = "coco/data/cleaned/parametros/parametros.xlsx" # Ruta de los parametros de cotizacion
ruta_cuotas = "coco/data/cleaned/cuotas/experiencia_global.xlsx" # Ruta de las cuotas de cotizacion
ruta_emisiones = "coco/data/cleaned/emisiones/emisiones.xlsx" # Ruta de las emisiones de cotizacion
ruta_historico_cotizaciones = "coco/data/cleaned/historico/cotizaciones.xlsx" # Ruta del historico de cotizaciones

# Bucket
nombre_bucket = "itam-analytics-grb"

# Lista de archivos en la base de datos
lista_archivos_base_datos = obtener_lista_nombre_bases(ruta_calculo, bucket_name)

# Cargamos base de parámetros
df_parametros = obtener_base_parametros(ruta_parametros, bucket_name)

# Cargamos base de cuotas
df_cuotas = obtener_base_cuotas(ruta_cuotas, bucket_name)

# Cargamos base de emisiones
df_emisiones = obtener_base_emisiones(ruta_emisiones, bucket_name)

# Cargamos historico de cotizaciones
df_hist_cotizaciones = obtener_base_historico(ruta_historico_cotizaciones, bucket_name)

# Cargamos bases de calculo y los guardamos en un diccionario y concatenamos las bases en uno solo
dfs_calculos = {}
for ruta in lista_archivos_base_datos:
    response = s3.get_object(Bucket=nombre_bucket, Key=ruta)
    content = response['Body'].read()
    df_calculo = pd.read_excel(BytesIO(content), engine='openpyxl')
    dfs_calculos[ruta] = df_calculo
# Concatenamos las bases de cálculo
df_calculo = pd.concat(dfs_calculos.values(), ignore_index=True)

# 2. Transformación de datos (los que van a ir en master_data: diccionario de cotización, memoria de cálculo)
s3 = boto3.client('s3')
ruta_dict = "coco/data/master_data/dict/"
ruta_memoria_calculo = "coco/data/master_data/memoria_calculo/"

dicts_contratantes = {}
ticket = len(df_hist_cotizaciones) + 1
# Proceso 1 "SAGEMAKER": Crear diccionario de cotización, memoria de cálculo e historial de cotizaciones y subir a master_data de S3
for contratante in df_parametros['Contratante'].unique():
    
    # Creamos el diccionario de parámetros para cada contratante
    df_calculo_contratante = df_calculo[df_calculo['Contratante'] == contratante].copy()
    
    dict_contratante = creacion_cotizacion_dict(df_parametros, contratante, ticket, df_calculo_contratante, df_emisiones, df_cuotas)
    df_dict_contratante = pd.DataFrame(dict_contratante)
    dicts_contratantes[contratante] = df_dict_contratante
    ticket += 1
    # Guardamos el diccionario como json
    ruta_dict_contratante = f'{ruta_dict}{contratante}.json'
    s3.put_object(Bucket=nombre_bucket, 
                  Key=ruta_dict_contratante, 
                  Body=json.dumps(dict_contratante, indent=2, ensure_ascii=False, default=str).encode('utf-8'),
                  ContentType='application/json')

    # Primer proceso: Subir a master: memoria de calculo,  la base de cotizaciones actualizada y el diccionario de cotización
    # Creamos memoria de cálculo
    fecha_corte = dict_contratante['Inicio'][0]
    contratante = dict_contratante['Contratante'][0]
    descuento = dict_contratante['Descuento'][0]
    rpf = dict_contratante['RPF'][0]
    memoria_calculo = generar_memoria_calculo(contratante, fecha_corte, df_parametros, df_calculo_contratante, df_cuotas, descuento, rpf)

    # Subir la memoria de cálculo a S3 coo csv
    ruta_memoria_calculo_completa = f'{ruta_memoria_calculo}{contratante}.csv'
    memoria_csv = memoria_calculo.to_csv(index=False)
    
    s3.put_object(
    Bucket=nombre_bucket,
    Key=ruta_memoria_calculo_completa,
    Body=memoria_csv.encode('utf-8'),
    ContentType='text/csv'
    )

# Historial de cotizaciones actualizado
df_dict_contratantes = pd.concat(dicts_contratantes.values(), ignore_index=True)
df_dict_contratantes['Tipo'] = np.where(df_dict_contratantes['Renovacion'] == 'Si', 'renovación', 'nuevo') # Cambiar por True/False
df_dict_contratantes['Fecha de Inicio'] =df_dict_contratantes['Inicio']
cols = ['Ticket', 'Fecha de Inicio', 'Mes', "Oficina", "Contratante", "Agente", "Prima", "Evento", "Tipo"]
df_dict_contratantes = df_dict_contratantes[cols]

df_hist_cotizaciones_actualizado = pd.concat([df_hist_cotizaciones, df_dict_contratantes], ignore_index=True)

# Subir el historial de cotizaciones actualizado a S3 como csv
ruta_hist_cotizaciones_actualizado = f'coco/data/master_data/historico/historial_cotizaciones_actualizado.csv'
hist_csv = df_hist_cotizaciones_actualizado.to_csv(index=False)
s3.put_object(
    Bucket=nombre_bucket,
    Key=ruta_hist_cotizaciones_actualizado,
    Body=hist_csv.encode('utf-8'),
    ContentType='text/csv'
)


{'ResponseMetadata': {'RequestId': '7FZJ0TQS2WEW5RMK',
  'HostId': 'jTXhkxkPNTuyaS7v1BGn+BhqvzR08foXmNSnkB0QQPLrzdzBfd1intGnwFsny0sdZDalUs27Cio=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'jTXhkxkPNTuyaS7v1BGn+BhqvzR08foXmNSnkB0QQPLrzdzBfd1intGnwFsny0sdZDalUs27Cio=',
   'x-amz-request-id': '7FZJ0TQS2WEW5RMK',
   'date': 'Wed, 28 May 2025 02:57:01 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"de6808dd9ec1b88013a7fd5c991e2c47"',
   'x-amz-checksum-crc32': '+AjURg==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"de6808dd9ec1b88013a7fd5c991e2c47"',
 'ChecksumCRC32': '+AjURg==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

### PIPELINE PDF

In [2]:
# Pipeline PDF
bucket_name = "itam-analytics-grb"
s3 = boto3.client("s3")


# Leemos todos los nombres de empresas de los jsons de cotización
ruta_dict = "coco/data/master_data/dict/"

# Obtenemos los nombres de las empresas
nombres_empresas = obtener_nombres_empresas(bucket_name, ruta_dict)

# Listas para regularización de campos
campos_float = ['SumaAsegurada', 'Comision', 'Ticket', 'RPF', 'NumRecibos', 
          'Descuento', 'Prima', 'EdadPromedio', 'SAMI', 'Asegurados']

campos_fecha = ['Inicio', 'Fin']





In [ ]:
for empresa in nombres_empresas:
    # Cargamos el diccionario de cotización de la empresa
    dict_empresa = cargar_dict_cotizacion(empresa, bucket_name)

    # Formateamos los campos del diccionario
    for campo in campos_float:
        dict_empresa = convertir_campo_a_float(dict_empresa, campo)

    for campo in campos_fecha:
        dict_empresa = convertir_campo_a_fecha(dict_empresa, campo)


    pdf_empresa = generar_pdf_cotizacion(bucket_name, dict_empresa)
    s3.upload_fileobj(pdf_empresa, bucket_name, f"coco/data/master_data/pdf/{empresa}.pdf")